<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-2/Insumos/MertonKMV_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paso de carga de librerías en R de todas las que se utilizarán

In [0]:
install.packages("quantmod")
install.packages("curl")
install.packages("data.table")
install.packages("RCurl")
install.packages("bitops")
install.packages("setInternet2")
install.packages("rootSolve")
install.packages("xml2")
library(rootSolve)
library(quantmod)
library(RCurl)
library(data.table)
library(xml2)
require(jsonlite) #para el getQuotes


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘setInternet2’ is not available (for R version 3.6.3)”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [0]:
Symbols<-c  ("WALMEX.MX")#tienen que ir en orden alfabético
length(Symbols)


start_date=Sys.Date()-3660 #fecha inicial

#Creación del objeto para guardar los datos
dataEnv<-new.env()

#obtener los datos
getSymbols.yahoo(Symbols,env=dataEnv,from=start_date)


#limpiarlos, alinearnos y quedarnos con el precio de cierre 



#muestra de datos
#Nos quedamos con los precios
stock_prices = na.omit(dataEnv$`WALMEX.MX`)
stock_prices=stock_prices[,4] #Close
tail(stock_prices[,])


[1] 1

Warning message:
“WALMEX.MX contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”


[1] "WALMEX.MX"

           WALMEX.MX.Close
2020-05-18           58.48
2020-05-19           56.42
2020-05-20           56.33
2020-05-21           55.84
2020-05-22           55.81
2020-05-25           55.68

Se deja el código de "webscrapping" hay que arreglarlo

In [0]:

#PARA CARGAR EL NÚMERO DE ACCIONES EN VENTA

getQuote <- function(ticks) {
  qRoot <- "https://query1.finance.yahoo.com/v7/finance/quote?fields=symbol,longName,regularMarketPrice,regularMarketChange,regularMarketTime,sharesOutstanding,bookValue&formatted=false&symbols="
  z <- fromJSON(paste(qRoot, paste(ticks, collapse=","), sep=""))
  z <- z$quoteResponse$result[,c("symbol", "regularMarketTime", "regularMarketPrice", "regularMarketChange", "longName", "sharesOutstanding", "bookValue" )]
  row.names(z) <- z$symbol
  z$symbol <- NULL
  names(z) <- c("Time", "Price", "Change", "Name", "Numeroacciones", "valorlibros")
  z$Time <- as.POSIXct(z$Time, origin = '1970-01-01 00:00:00')
  return(z)
}


ticks=Symbols

stqt=getQuote(Symbols)

stqt

#PARA CARGAR LOS ESTADOS FINANCIEROS Y FLUJOS DE EFECTIVO
url <- "https://finance.yahoo.com/quote/"
scrapy_stocks <- function(stock){
  if ("rvest" %in% installed.packages()) {
    library(rvest)
  }else{
    install.packages("rvest")
    library(rvest)
  }
  for (i in 1:length(stock)) {
    tryCatch(
      {
        # Income Statement
        # financial <- paste0(url,stock[i],"/financials?p=",stock[i])
        # wahis.session <- html_session(financial)                                
        # p <-    wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
        #   .[[1]] %>% html_children() %>% html_nodes("div")
        # suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
        #                    html_text() %>% gsub(pattern = ",",replacement = "") %>% 
        #                    gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        # Headers <- wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
        #   .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        # Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,88,4)]
        # IS <- data.frame(Breakdown = Concepts[-c(4,16,19)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        # names(IS) <- Headers
        # temp1 <- IS
        
        # Balance Sheet
        bsheet <- paste0(url,stock[i],"/balance-sheet?p=",stock[i])
        wahis.session <- html_session(bsheet)
        p <-    wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
          .[[1]] %>% html_children() %>% html_nodes("div")
        suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
                           html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric()) 
        Headers <- wahis.session %>% 
          html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
          .[[1]] %>% html_nodes("span") %>% html_text() %>% head(5)
        Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,175,4)]
        BS <- data.frame(Breakdown = Concepts[-c(1:3,11,12,22:24,32,39)], matrix(Values[!is.na(Values)], ncol = 4, byrow = T))
        names(BS) <- Headers
        temp2 <- BS
        
        # Cash flows
        # cash_f <- paste0(url,stock[i],"/cash-flow?p=",stock[i])
        # wahis.session <- html_session(cash_f)
        # p <-    wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]/div[2]') %>%
        #   .[[1]] %>% html_children() %>% html_nodes("div")
        # suppressWarnings(Values <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 3] %>% 
        #                    html_text() %>% gsub(pattern = ",",replacement = "") %>% gsub(pattern = "-",replacement = 0) %>% as.numeric())
        # Headers <- wahis.session %>% 
        #   html_nodes(xpath = '//*[@id="Col1-1-Financials-Proxy"]/section/div[4]/div[1]/div[1]') %>% 
        #   .[[1]] %>% html_nodes("span") %>% html_text() %>% head()
        # Concepts <- p[sapply(p, function(x){length(x %>% html_attrs())}) == 2] %>% html_text() %>% .[seq(2,133,4)]
        # CF <- data.frame(Breakdown = Concepts[-c(1,13,20,30)], matrix(Values[!is.na(Values)], ncol = 5, byrow = T))
        # names(CF) <- Headers
        # temp3 <- CF
        
        # Shares Outstanding
        stock_url <- paste0(url,stock[i],"/key-statistics?p=",stock[i])
        SO <- html_session(stock_url) %>%
          html_nodes(xpath = '//*[@id="Col1-0-KeyStatistics-Proxy"]/section/div[2]/div[2]/div/div[2]/div/table/tbody/tr[3]/td[2]') %>% 
          html_text() %>% gsub(pattern = "[A-z]",replacement = "") %>% as.numeric()
        temp4 <- SO
        
        assign(paste0(stock[i],'.f'),value = list(IS = temp1,BS = temp2,CF = temp3, SO = temp4),envir = parent.frame())
        
      },
      error = function(cond){
        message(stock[i], "Give error ",cond)
      }
    )
  }
}

#stock=Symbols
#scrapy_stocks(Symbols) 
#attach(WALMEX.MX.f)
#WALMEX.MX.f$BS
#WALMEX.MX.f$IS
#WALMEX.MX.f$CF
#BS

stqt$Numeroacciones


,Time,Price,Change,Name,Numeroacciones,valorlibros
,<dttm>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
WALMEX.MX,2020-05-25 19:59:51,55.68,-0.1300011,"Wal-Mart de Mexico, S.A.B. de C.V.",17461399552,9.327


[1] 17461399552

In [0]:
TSE <- "Total stockholders' equity"
TL <- "Total Liabilities"
# dt <- colnames(BS)[2] # ÚLTIMA FECHA DISPONIBLE

dt="12/31/2019"

#K_0=BS[33,1]/1000 #Total Stockholder Equity VALOR DE LA EMPRESA
#en millones https://finance.yahoo.com/quote/WALMEX.MX/balance-sheet?p=WALMEX.MX
#Total stockholders' equity
#En millones
# K_0=BS[BS$Breakdown==TSE, dt]*1000/1000000
K_0=168144767*1000/1000000
#nrow(BS)
#D_0=BS[24,1]/1000 #Total Liabilities DEUDA ACTUAL
#En millones
# D_0=BS[BS$Breakdown==TL, dt]*1000/1000000
D_0=181027502*1000/1000000

#Shares Outstanding (Sexpresa en miles de millones en la página)
#En millones
Numeroacciones=stqt$Numeroacciones/1000000

dt=strptime(dt, "%m/%d/%Y")
dt=format(dt, "%Y-%m")
dt
#no esta sentencia entonces mi ultima fecha fue tanto 
n=nrow(stock_prices[paste("/",dt,sep="")]) #NUMERO DE FILAS A ANALIZAR PARA EL ÚLTIMO DATO

A_0=stock_prices[paste("/",dt,sep="")][n] #valores actuales
print("precio actual de la acción")
print(A_0)

#A_0=A_0*stqt$Numeroacciones/1000000 #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
A_0=A_0*Numeroacciones #VALOR ACTUAL DE LAS ACCIONES VALOR DE ACCIÓN POR ACCIONES DISPONIBLES
print("Ratio entre A0 y K0")
A_0/K_0

A=stock_prices[paste("/",dt,sep="")]  #VECTOR DE PRECIOS 

vol_A=sqrt(var(log(as.matrix(A[2:n])/as.matrix(A[1:(n-1)]))))*sqrt(252) #VOLATILIDAD ANUAL

#Valor del CETE
r=0.0717
T=1

A_0
K_0
vol_A
D_0
r
T


[1] "2019-12"

[1] "precio actual de la acción"
           WALMEX.MX.Close
2019-12-31           54.18
[1] "Ratio entre A0 y K0"


           WALMEX.MX.Close
2019-12-31        5.626453

           WALMEX.MX.Close
2019-12-31        946058.6

[1] 168144.8

,WALMEX.MX.Close
WALMEX.MX.Close,0.237323


[1] 181027.5

[1] 0.0717

[1] 1

In [0]:
x=c(D_0,vol_A)

#Función para estimar vol_k y D
G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

#K_0=164370

ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

print("Deuda")
D=ss$root[1]
D
print("Vol k")
vol_k=ss$root[2]
vol_k
print("D_T/D_0")
ss$root[1]/D_0
print("Vol_A/Vol_K")
vol_A/ss$root[2]
print("evaluacion en raíces")
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
print("PD MERTON")
PD_merton


Warning message in stode(y, times, func, parms = parms, ...):
“steady-state not reached”


[1] "Deuda"


[1] 0

[1] "Vol k"


[1] 1.335286

...

,WALMEX.MX.Close
WALMEX.MX.Close,0.1777319


[1] "evaluacion en raíces"


           WALMEX.MX.Close
2019-12-31        777913.9
2019-12-31             0.0

[1] "PD MERTON"


[1] 0

In [0]:
#Sólo estimar la volatilidad dada la deuda
#Probabilidad de Incumplimiento

Vk = function(x) 
{
  vol_k <- x
  ((vol_k/vol_A)*(K_0/A_0))-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
}

#Definir una Deuda
D=D_0
x=vol_A #semilla de la vol_k
ss=multiroot(Vk,x,positive = TRUE) #RESOLVER EL SISTEMA.
print("vol_k")
vol_k=ss$root
vol_k
print("Ratio vol_k/vol_A")
ss$root[1]/vol_A
print("valor Vk bajo solución")
Vk(vol_k)

PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))

print("Deuda estimada")
D
print("Deuda sencilla")
D_0
print("Vol estimada")
vol_k
print("Vol semilla")
vol_A
print("PD Merton")
PD_merton



[1] "vol_k"


[1] 0.8978517

[1] "Ratio vol_k/vol_A"


,WALMEX.MX.Close
WALMEX.MX.Close,3.783249


[1] "valor Vk bajo solución"


...

[1] 0.8978517

[1] "Vol semilla"


,WALMEX.MX.Close
WALMEX.MX.Close,0.237323


[1] "PD Merton"


[1] 0.3275962

#KMV

In [0]:
print("A_0:")
A_0
print("Valor de K_0")
K_0
print("Valor de vol_A")
vol_A
print("Valor de Deuda Actual")
D_0
print("Tasa libre de riesgo")
r
print("Temporalidad")
T


#Función para estimar vol_k y D
Gkmv = function(x) 
{
  vol_k <- x[2]
  K_0 <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
    A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

D=D_0

x=c(K_0,vol_A)

ss=multiroot(Gkmv,x,positive = TRUE) #RESOLVER EL SISTEMA.

K_e=ss$root[1]
vol_k=ss$root[2]
print("Cociente K_0/D_0")
K_e/D_0
print("Cociente vol_K/vol_A")
ss$root[2]/vol_A
print("Valor de solución en función:")
Gkmv(c(K_e,vol_k))

DI=(K_e-D)/(vol_k*K_0)
print("DI")
DI
EDF=pnorm(-DI)
print("EDF Normal")
EDF
 


[1] "A_0:"


           WALMEX.MX.Close
2019-12-31        946058.6

[1] "Valor de K_0"


[1] 168144.8

[1] "Valor de vol_A"


...

           WALMEX.MX.Close
2019-12-31               0
2019-12-31               0

[1] "DI"


[1] 27.56084

[1] "EDF Normal"


[1] 1.640641e-167